* The Unsloth template for this notebook - https://colab.research.google.com/drive/1T5-zKWM_5OD21QHwXHiV9ixTRR7k3iB9?usp=sharing#scrollTo=IqM-T1RTzY6C

* The Llama-3.2-3B-Instruct model - https://huggingface.co/unsloth/Llama-3.2-3B-Instruct

In [1]:
%%capture
!pip install unsloth "xformers==0.0.28.post2" --quiet
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --quiet
!pip install datasets mlflow --quiet

In [2]:
from datetime import datetime
import mlflow
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import get_chat_template

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 1994,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [16]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    system_message = None
    )

In [17]:
def formatting_prompts_func(examples):
    convos = examples["task"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [18]:
from datasets import load_dataset
dataset = load_dataset('winterForestStump/cbr_bonds_info_detector')

In [19]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [20]:
print(dataset['train'][0]['text'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

#Instruction: You have to extract securities registration numbers from the provided text. #Provided text: Банк России 16.11.2023 принял решение о государственной регистрации выпусков неконвертируемых процентных бездокументарных облигаций серий ЗО25-Д, ЗО26-Д, ЗО27-Д, ЗО28-Д и ЗО29-Д акционерного общества «Государственная транспортная лизинговая компания» (Ямало-Ненецкий автономный округ), размещаемых путем открытой подписки. Выпускам ценных бумаг присвоены регистрационные номера 4-04-32432-H, 4-05-32432-H, 4-06-32432-H, 4-07-32432-H и 4-08-32432-H соответственно.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

4-04-32432-H, 4-05-32432-H, 4-06-32432-H, 4-07-32432-H, 4-08-32432-H<|eot_id|>


In [21]:
name = "Llama-3.2-3B-Instruct fine-tuning"

mlflow.set_experiment(name)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = dataset['test'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=3,
        #max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1994,
        evaluation_strategy = "steps",
        output_dir = "outputs",
        run_name=f"{name}-{datetime.now().strftime('%Y-%m-%d-%H-%M-%s')}",
        report_to = "mlflow",
    ),
)

2024/12/01 14:18:23 INFO mlflow.tracking.fluent: Experiment with name 'Llama-3.2-3B-Instruct fine-tuning' does not exist. Creating a new experiment.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/119 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

In [22]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.635 GB of memory reserved.


In [23]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 119 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 45
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,Validation Loss
1,2.066700,2.302467
2,3.176500,2.263669
3,2.359300,2.114578
4,3.090200,1.950215
5,2.871600,1.787136
6,1.706200,1.624680
7,1.730500,1.500655
8,1.633800,1.396631
9,1.540800,1.302518
10,1.251000,1.207423


In [24]:
!zip -r mlruns.zip /content/mlruns

  adding: content/mlruns/ (stored 0%)
  adding: content/mlruns/.trash/ (stored 0%)
  adding: content/mlruns/111638724200738015/ (stored 0%)
  adding: content/mlruns/111638724200738015/2d4b828fbc6d43d595d0e6c7bb2a64f3/ (stored 0%)
  adding: content/mlruns/111638724200738015/2d4b828fbc6d43d595d0e6c7bb2a64f3/tags/ (stored 0%)
  adding: content/mlruns/111638724200738015/2d4b828fbc6d43d595d0e6c7bb2a64f3/tags/mlflow.user (stored 0%)
  adding: content/mlruns/111638724200738015/2d4b828fbc6d43d595d0e6c7bb2a64f3/tags/mlflow.source.type (stored 0%)
  adding: content/mlruns/111638724200738015/2d4b828fbc6d43d595d0e6c7bb2a64f3/tags/mlflow.runName (stored 0%)
  adding: content/mlruns/111638724200738015/2d4b828fbc6d43d595d0e6c7bb2a64f3/tags/mlflow.source.name (deflated 5%)
  adding: content/mlruns/111638724200738015/2d4b828fbc6d43d595d0e6c7bb2a64f3/params/ (stored 0%)
  adding: content/mlruns/111638724200738015/2d4b828fbc6d43d595d0e6c7bb2a64f3/params/prediction_loss_only (stored 0%)
  adding: content/

In [25]:
from google.colab import files
files.download("mlruns.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

676.5596 seconds used for training.
11.28 minutes used for training.
Peak reserved memory = 6.357 GB.
Peak reserved memory for training = 3.722 GB.
Peak reserved memory % of max memory = 43.104 %.
Peak reserved memory for training % of max memory = 25.237 %.


In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token='') # token = "hf_..."

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = '') # token = "hf_..."

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = '') # token = "hf_..."


In [ ]:
# Save to multiple GGUF options - much faster if you want multiple!
if True:
    model.push_to_hub_gguf(
        "winterForestStump/Llama-3.2-3B-Instruct-CBR_Bonds_info", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.2G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.15 out of 12.67 RAM for saving.


100%|██████████| 28/28 [00:02<00:00, 11.24it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving winterForestStump/Llama-3.2-3B-Instruct-CBR_Bonds_info/pytorch_model-00001-of-00002.bin...
Unsloth: Saving winterForestStump/Llama-3.2-3B-Instruct-CBR_Bonds_info/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


...

unsloth.Q5_K_M.gguf:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/winterForestStump/Llama-3.2-3B-Instruct-CBR_Bonds_info


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


Saved Ollama Modelfile to https://huggingface.co/winterForestStump/Llama-3.2-3B-Instruct-CBR_Bonds_info
